## 論文読解

以下の論文を読み問題に答えてください。CNNを使った物体検出（Object Detection）の代表的な研究です。


[8]Ren, S., He, K., Girshick, R., Sun, J.: Faster r-cnn: Towards real-time object detection with region proposal networks. In: Advances in neural information processing systems. (2015) 91–99


https://arxiv.org/pdf/1506.01497.pdf

## 問題

それぞれについてJupyter Notebookにマークダウン形式で記述してください。

(1) 物体検出の分野にはどういった手法が存在したか。


(2) Fasterとあるが、どういった仕組みで高速化したのか。


(3) One-Stageの手法とTwo-Stageの手法はどう違うのか。


(4) RPNとは何か。


(5) RoIプーリングとは何か。


(6) Anchorのサイズはどうするのが適切か。


(7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。


(8) （アドバンス課題）Faster R-CNNよりも新しい物体検出の論文では、Faster R-CNNがどう引用されているか。



## 条件

・答える際は論文のどの部分からそれが分かるかを書く。

・必要に応じて先行研究（引用されている論文）も探しにいく。最低2つは他の論文を利用して回答すること。

・論文の紹介記事を見ても良い。ただし、答えは論文内に根拠を探すこと。


## (1) 物体検出の分野にはどういった手法が存在したか。

SPPnetやFast R-CNN

該当 : Advances like SPPnet [1] and Fast R-CNN [2] have reduced the running time of these detection networks, exposing region

## (2) Fasterとあるが、どういった仕組みで高速化したのか。

Selective searchではなくRPNというCNN構造を用い、end-to-endでトレーニングして高速化


該当 : In this work, we introduce a Region Proposal Network (RPN) that shares full-image
convolutional features with the detection network, thus enabling nearly cost-free region proposals. An RPN is a fully convolutional
network that simultaneously predicts object bounds and objectness scores at each position. The RPN is trained end-to-end to
generate high-quality region proposals, which are used by Fast R-CNN for detection. We further merge RPN and Fast R-CNN
into a single network by sharing their convolutional features—using the recently popular terminology of neural networks with
“attention” mechanisms, the RPN component tells the unified network where to look. For the very deep VGG-16 model [3],
our detection system has a frame rate of 5fps (including all steps) on a GPU, while achieving state-of-the-art object detection
accuracy on PASCAL VOC 2007, 2012, and MS COCO datasets with only 300 proposals per image. In ILSVRC and COCO
2015 competitions, Faster R-CNN and RPN are the foundations of the 1st-place winning entries in several tracks. Code has been
made publicly available.


① ある矩形が物体なのか背景なのかを学習 ②①で検出した場所に、具体的に何が写っているのかを学習
という2段構造  ![](https://miro.medium.com/max/509/1*BhrLkEQdxVnE8RJ9iY91kA.jpeg)

[参考](https://medium.com/lsc-psd/faster-r-cnn%E3%81%AB%E3%81%8A%E3%81%91%E3%82%8Brpn%E3%81%AE%E4%B8%96%E7%95%8C%E4%B8%80%E5%88%86%E3%81%8B%E3%82%8A%E3%82%84%E3%81%99%E3%81%84%E8%A7%A3%E8%AA%AC-dfc0c293cb69)

end-to-endとは入力と出力のみで、途中の過程にある全てのステップを含めて学習してしまうこと


![](https://teratail.storage.googleapis.com/uploads/contributed_images/ede853ce56da4b71801658d7fbf146db.png)


上記画像を例にすると画像からいろいろなタスクを踏んで文字認識をするのが一般的な手法ですがEndtoEndの場合画像から文字認識まで一気に学習させる手法

## (3) One-Stageの手法とTwo-Stageの手法はどう違うのか。

One-Stage : 領域提案部分が分離されていない

Two-Stage : 領域提案部分が分離されておらず、処理が一度で済むようになっている

該当 : One-Stage Detection vs. Two-Stage Proposal + Detection. The OverFeat paper [9] proposes a detection
method that uses regressors and classifiers on sliding
windows over convolutional feature maps. OverFeat
is a one-stage, class-specific detection pipeline, and ours
is a two-stage cascade consisting of class-agnostic proposals and class-specific detections. In OverFeat, the
region-wise features come from a sliding window of
one aspect ratio over a scale pyramid. These features
are used to simultaneously determine the location and
category of objects. In RPN, the features are from
square (3×3) sliding windows and predict proposals
relative to anchors with different scales and aspect
ratios. Though both methods use sliding windows, the
region proposal task is only the first stage of Faster RCNN—the downstream Fast R-CNN detector attends
to the proposals to refine them. In the second stage of
our cascade, the region-wise features are adaptively
pooled [1], [2] from proposal boxes that more faithfully cover the features of the regions. We beli


## (4) RPNとは何か。

1.特徴マップを入力しそれぞれオブジェクトネススコアを保持した長方形の領域提案のセットを出力する
＝画像が物体かどうか表すスコア計算

2.領域提案を生成するために、小さな畳み込みネットワークを共有された畳み込み層によって出力された畳み込み特徴マップにスライドさせる

該当 : A Region Proposal Network (RPN) takes an image (of any size) as input and outputs a set of rectangular object proposals, each with an objectness score. We model this process with a fully convolutional network [7], which we describe in this section. Because our ultimate goal is to share computation with a Fast R-CNN object detection network [2], we assume that both nets share a common set of convolutional layers. In our experiments, we investigate the Zeiler and Fergus model [32] (ZF), which has 5 shareable convolutional layers and the Simonyan and Zisserman model [3] (VGG-16), which has 13 shareable convolutional layers. To generate region proposals, we slide a small network over the convolutional feature map output by the last shared convolutional layer. This small network takes as input an n × n spatial window of the input convolutional feature map. Each sliding window is mapped to a lower-dimensional feature (256-d for ZF and 512-d for VGG, with ReLU [33] following). This feature is fed into two sibling fullyconnected layers—a box-regression layer (reg) and a box classification layer (cls). We use n = 3 in this paper, noting that the effective receptive field on the input image is large (171 and 228 pixels for ZF and VGG, respectively). This mini-network is illustrated at a single position in Figure 3 (left). Note that because the mini-network operates in a sliding-window fashion, the fully-connected layers are shared across all spatial locations. This architecture is naturally implemented with an n×n convolutional layer followed by two sibling 1 × 1 convolutional layers (for reg and cls, respectively).


![](https://news.mynavi.jp/article/cv_future-55/images/002.jpg)

## (5) RoIプーリングとは何か。

畳み込みを行った特徴マップからMaxプーロングを使用しハイパーパラメーターで固定した小さな特徴マップに変換


該当 : The RoI pooling layer uses max pooling to convert the
features inside any valid region of interest into a small feature map with a fixed spatial extent of H × W (e.g., 7 × 7),
where H and W are layer hyper-parameters that are independent of any particular RoI. In this paper, an RoI is a
rectangular window into a conv feature map. Each RoI is
defined by a four-tuple (r, c, h, w) that specifies its top-left
corner (r, c) and its height and width (h, w).
　RoI max pooling works by dividing the h × w RoI window into an H × W grid of sub-windows of approximate
size h/H × w/W and then max-pooling the values in each
sub-window into the corresponding output grid cell. Pooling is applied independently to each feature map channel,
as in standard max pooling. The RoI layer is simply the
special-case of the spatial pyramid pooling layer used in
SPPnets [11] in which there is only one pyramid level. We
use the pooling sub-window calculation given in [11].



![](https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F139809%2F0cdf39ad-ca31-5377-87c3-089bf78dea81.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&w=1400&fit=max&s=ba59c7bd7c4a9b41c3f8c2186c760a8b)

[Fast R-CNN 論文](https://www.cv-foundation.org/openaccess/content_iccv_2015/papers/Girshick_Fast_R-CNN_ICCV_2015_paper.pdf)

## (6) Anchorのサイズはどうするのが適切か。

・デフォルトでは3つのスケール、3つのアスペクト比、計9種類を使用

・128^2, 256^2, 512^2 ３つのスケール、1:1, 1:2, 2:1 ３つのアスペクト比合計9種類


該当 : anchors. An anchor is centered at the sliding window
in question, and is associated with a scale and aspect
ratio (Figure 3, left). By default we use 3 scales and
3 aspect ratios, yielding k = 9 anchors at each sliding
position. For a convolutional feature map of a size
W × H (typically ∼2,400), there are W Hk anchors in
total.


該当 : For anchors, we use 3 scales with box areas of 128^2,
256^2, and 512^2 pixels, and 3 aspect ratios of 1:1, 1:2,
and 2:1. These hyper-parameters are not carefully chosen for a particular dataset, and we provide ablation
experiments on their effects in the next section. As discussed, our solution does not need an image pyramid
or filter pyramid to predict regions of multiple scales,
saving considerable running time. Figure 3 (right)
shows the capability of our method for a wide range
of scales and aspect ratios. Table 1 shows the learned
average proposal size for each anchor using the ZF
net. We note that our algorithm allows predictions
that are larger than the underlying receptive field.
Such predictions are not impossible—one may still
roughly infer the extent of an object if only the middle
of the object is visible

## (7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。


PASCAL VOCやMS COCO mAPという指標値で評価


Fast R-CNNと比較している。Fast R-CNNに置いては、39.3% mAP@0.5の精度を出した、Faster R-CNNに置いては42.1% mAP@0.5 and 21.5% mAP@[.5, .95]の精度を出した。mAP@0.5においては2.8%、mAP@[.5, .95]に置いては2.2%高い結果である。

該当 : In Table 11 we first report the results of the Fast
R-CNN system [2] using the implementation in this
paper. Our Fast R-CNN baseline has 39.3% mAP@0.5
on the test-dev set, higher than that reported in [2].
We conjecture that the reason for this gap is mainly
due to the definition of the negative samples and also
the changes of the mini-batch sizes. We also note that
the mAP@[.5, .95] is just comparable.
Next we evaluate our Faster R-CNN system. Using
the COCO training set to train, Faster R-CNN has
42.1% mAP@0.5 and 21.5% mAP@[.5, .95] on the
COCO test-dev set. This is 2.8% higher for mAP@0.5
and 2.2% higher for mAP@[.5, .95] than the Fast RCNN counterpart under the same protocol (Table 11).
This indicates that RPN performs excellent for improving the localization accuracy at higher IoU thresholds. Using the COCO trainval set to train, Faster RCNN has 42.7% mAP@0.5 and 21.9% mAP@[.5, .95] on
the COCO test-dev set. Figure 6 shows some results
on the MS COCO test-dev set.



## (8) （アドバンス課題）Faster R-CNNよりも新しい物体検出の論文では、Faster R-CNNがどう引用されているか。

YOLOなどの高速で物体認識などの論文で識別精度などで比較されている